In [ ]:
from utils.cn import Linear
from utils.tensor import Tensor
import numpy as np

m = Linear(n_in=600, n_out=600, bias=True)

x = Tensor(np.random.randn(10, 600))

y = m(x)
y.backward()

print(y)
print(m.parameter.grad)
print(m.bias.grad)

In [85]:
import numpy as np

arr = np.zeros([2,3,4,5,6])
arr = np.array([12])

shape = arr.shape

# a = shape[:2]
# b = shape[2:4]
# c = shape[4:]
# f = a + b + c
# f
shape[:0], shape[0], shape[0+1:]

((), 1, ())

In [39]:
def flatten(arr, start_dim: int = 0, end_dim: int = -1):
    shape = arr.shape

    if start_dim < 0:
        start_dim = len(shape) + start_dim 
    if end_dim < 0:
        end_dim = len(shape) + end_dim + 1
    
    # Compose new shape
    start_shape = shape[:start_dim]
    flatten_dim = np.prod(shape[start_dim:end_dim])
    end_shape = shape[end_dim:]


    final_shape = start_shape + (flatten_dim,) + end_shape

    flat = np.reshape(arr, final_shape)
    return flat

shaped = flatten(arr, 1, -1)
print(shaped.shape)

b = shaped.reshape(shape)
b.shape

(2, 360)


(2, 3, 4, 5, 6)

In [ ]:
import numpy as np
arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(arr.size)


In [90]:
np.append(np.ones(3), np.ones(3)).reshape(2,3).T

array([[1., 1.],
       [1., 1.],
       [1., 1.]])

In [ ]:
import numpy as np

#Im2Col

d = 2
im = np.arange(d*5*5).reshape((d,5,5))
kernel_size = (3,3)
print(im)
print("\n")

# Calculate output size
h_out = im.shape[1] - kernel_size[0] + 1
w_out = im.shape[2] - kernel_size[1] + 1

# Use stride tricks to create a sliding window view
# The shape will be (d, h_out, w_out, kH, kW)
shape = (d, h_out, w_out, kernel_size[0], kernel_size[1])
strides = (
    im.strides[0],
    im.strides[1],
    im.strides[2],
    im.strides[1],
    im.strides[2]
)
windows = np.lib.stride_tricks.as_strided(im, shape=shape, strides=strides)

# Move channel to the last dim and reshape to a 2D array as desired
col = windows.transpose(1, 2, 0, 3, 4).reshape(h_out * w_out, d * kernel_size[0] * kernel_size[1]).T
col

[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]

 [[25 26 27 28 29]
  [30 31 32 33 34]
  [35 36 37 38 39]
  [40 41 42 43 44]
  [45 46 47 48 49]]]




array([[ 0,  1,  2,  5,  6,  7, 10, 11, 12],
       [ 1,  2,  3,  6,  7,  8, 11, 12, 13],
       [ 2,  3,  4,  7,  8,  9, 12, 13, 14],
       [ 5,  6,  7, 10, 11, 12, 15, 16, 17],
       [ 6,  7,  8, 11, 12, 13, 16, 17, 18],
       [ 7,  8,  9, 12, 13, 14, 17, 18, 19],
       [10, 11, 12, 15, 16, 17, 20, 21, 22],
       [11, 12, 13, 16, 17, 18, 21, 22, 23],
       [12, 13, 14, 17, 18, 19, 22, 23, 24],
       [25, 26, 27, 30, 31, 32, 35, 36, 37],
       [26, 27, 28, 31, 32, 33, 36, 37, 38],
       [27, 28, 29, 32, 33, 34, 37, 38, 39],
       [30, 31, 32, 35, 36, 37, 40, 41, 42],
       [31, 32, 33, 36, 37, 38, 41, 42, 43],
       [32, 33, 34, 37, 38, 39, 42, 43, 44],
       [35, 36, 37, 40, 41, 42, 45, 46, 47],
       [36, 37, 38, 41, 42, 43, 46, 47, 48],
       [37, 38, 39, 42, 43, 44, 47, 48, 49]])

In [ ]:
import numpy as np

# This is still roughly 10-20 times slower than using np.strides from above, but I refuse to use that implementation
## since this part of the code now takes up relatively very little time (I hope)
## It might also eat up all the memory of the computer. You never know. Seems to scale fine with input dim but less fine wiht spacial img size
# It might be smarter to loop over rows instead of columns? It is symmetrical within each image after all
## The stride code is disgusting

d = 2
im = np.arange(d*5*5).reshape((d,5,5))
start_pos = (1,2)
kernel_size = (3,3)
print(im)
print("\n")

h = list(range(im.shape[1] - kernel_size[0] + 1))
w = list(range(im.shape[2] - kernel_size[1] + 1))

col = []
for i in h:
    for j in w:
        patch = im[:, i:i + kernel_size[0], j:j + kernel_size[1]]
        reshaped = patch.reshape(kernel_size[0]*kernel_size[1] * d)
        col.append(reshaped) #Using Python append and not np.append (O(1) vs O(N^2))

col = np.concat(col)
col = col.reshape(len(h)*len(w), kernel_size[0]*kernel_size[1] * d).T
print(col)

[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]

 [[25 26 27 28 29]
  [30 31 32 33 34]
  [35 36 37 38 39]
  [40 41 42 43 44]
  [45 46 47 48 49]]]


[[ 0  1  2  5  6  7 10 11 12]
 [ 1  2  3  6  7  8 11 12 13]
 [ 2  3  4  7  8  9 12 13 14]
 [ 5  6  7 10 11 12 15 16 17]
 [ 6  7  8 11 12 13 16 17 18]
 [ 7  8  9 12 13 14 17 18 19]
 [10 11 12 15 16 17 20 21 22]
 [11 12 13 16 17 18 21 22 23]
 [12 13 14 17 18 19 22 23 24]
 [25 26 27 30 31 32 35 36 37]
 [26 27 28 31 32 33 36 37 38]
 [27 28 29 32 33 34 37 38 39]
 [30 31 32 35 36 37 40 41 42]
 [31 32 33 36 37 38 41 42 43]
 [32 33 34 37 38 39 42 43 44]
 [35 36 37 40 41 42 45 46 47]
 [36 37 38 41 42 43 46 47 48]
 [37 38 39 42 43 44 47 48 49]]


In [ ]:
from utils.dataloader import DataLoader, train_dataset, test_dataset
batch_size = 34
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=False)

len(test_loader.dataset)
test_loader.dataset.y

In [ ]:
from utils import candle

candle.is_grad_enabled()

with candle.no_grad():
    print(candle.is_grad_enabled())

In [ ]:
import numpy as np

b = np.array([[1, 2, 3], [4, 5, 6]])
f = np.array([[7, 8], [9, 10]])

b.T @ f

In [45]:
False and True

False

In [43]:
if not False:
    print("False")

False


In [ ]:
(f.T @ b).T

In [ ]:
dict_ = {"side": "test"}

try:
    start_index = dict_["start_index"]
    end_index = dict_["end_index"]
    matrix_side = dict_["matrix"]
except:
    start_index = None
    end_index = None
    matrix_side = None

side = dict_["side"]

if "matrix" in dict_:
    print("test1")

if 2 != None:
    print("test2")

In [ ]:
import numpy as np

# Create two 3D matrices (tensors)
# For valid matrix multiplication with shape (a, b, c) @ (a, c, d) -> (a, b, d)
A = np.random.randint(0, 10, (2, 3, 4))
B = np.random.randint(0, 10, (2, 4, 5))

# Multiply each (i)th 3x4 with (i)th 4x5 -> each result (i) is 3x5. All dims used.
C = np.empty((2, 3, 5), dtype=int)
for i in range(A.shape[0]):
    C[i] = A[i] @ B[i]

print("A.shape:", A.shape)
print("B.shape:", B.shape)
print("C.shape:", C.shape)
print(C)
